In [1]:
# Import Required Libraries
import os
import pandas as pd
from datetime import datetime, timedelta
import findspark

findspark.init()



# PySpark imports
try:
    from pyspark.sql import SparkSession
    pyspark_available = True
    print("PySpark is available!")
except ImportError:
    print("PySpark not found. Please install with: pip install pyspark")
    pyspark_available = False




PySpark is available!


In [2]:
if pyspark_available:
    # Create SparkSession with custom configuration
    spark = SparkSession.builder \
        .appName("PySpark") \
        .config("spark.sql.adaptive.enabled", "true") \
        .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
        .config("spark.driver.memory", "8g") \
        .config("spark.executor.memory", "4g") \
        .getOrCreate()
    
    # Set log level to reduce verbose output
    spark.sparkContext.setLogLevel("WARN")
    
    print("✓ SparkSession created successfully!")
    print(f"Spark Version: {spark.version}")
    print(f"Application Name: {spark.sparkContext.appName}")
    print(f"Master: {spark.sparkContext.master}")
    
    # Check available cores and memory
    print(f"Default Parallelism: {spark.sparkContext.defaultParallelism}")
    
else:
    print("Cannot proceed without PySpark. Please install PySpark first.")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/26 15:19:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


✓ SparkSession created successfully!
Spark Version: 3.5.0
Application Name: PySpark
Master: local[*]
Default Parallelism: 4


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import monotonically_increasing_id, concat, lit


file_path = "./students_unstructured.json"

spark = SparkSession.builder.appName("RemoveColumn").getOrCreate()

students_df = spark.read.json(file_path)
students_no_exam_scores = students_df.drop("Exam_Score")
students_with_ids = students_no_exam_scores.withColumn(
    "Student_ID",
    (monotonically_increasing_id() + 1)
)
students_with_ids.show(5)
students_with_ids.coalesce(1).write.mode("overwrite").json("./data/students_no_scores.json")

spark.stop()


25/11/26 15:19:03 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+-------------------+----------+------------------+--------------------------+-------------+------+-------------+---------------+---------------------+----------------+------------------------+--------------------+--------------+-----------------+---------------+-----------+-----------+---------------+-----------------+---------------+----------+
|Access_to_Resources|Attendance|Distance_from_Home|Extracurricular_Activities|Family_Income|Gender|Hours_Studied|Internet_Access|Learning_Disabilities|Motivation_Level|Parental_Education_Level|Parental_Involvement|Peer_Influence|Physical_Activity|Previous_Scores|School_Type|Sleep_Hours|Teacher_Quality|Tutoring_Sessions|_corrupt_record|Student_ID|
+-------------------+----------+------------------+--------------------------+-------------+------+-------------+---------------+---------------------+----------------+------------------------+--------------------+--------------+-----------------+---------------+-----------+-----------+---------------